In [23]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
import seaborn as sns


In [2]:
df = pd.read_csv("train_data.csv", encoding = "ISO-8859-1")
ff = pd.read_csv("test_data.csv", encoding = "ISO-8859-1")

In [3]:
df.head()

,message,food,recharge,support,reminders,travel,nearby,movies,casual,other
0,7am everyday,F,F,F,T,F,F,F,F,F
1,chocolate cake,T,F,F,F,F,F,F,F,F
2,closed mortice and tenon joint door dimentions,F,F,T,F,F,F,F,F,F
3,train eppo kelambum,F,F,F,F,T,F,F,F,F
4,yesterday i have cancelled the flight ticket,F,F,F,F,T,F,F,F,F


In [4]:
ff.iloc[0]['message']

u'Nearest metro station'

In [5]:
df_for_stats = df.drop(['message'], axis=1)
counts = []
categories = list(df_for_stats.columns.values)
for i in categories:
    count=0
    for x in df_for_stats[i]:
        if x=='T':
            count=count+1
    counts.append((i,count))
            
       #counts.append((i, df_for_stats[i].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number_of_comments'])
df_stats

,category,number_of_comments
0,food,3979
1,recharge,3087
2,support,1847
3,reminders,7331
4,travel,13504
5,nearby,4591
6,movies,4004
7,casual,6055
8,other,2989


In [6]:
df_for_stats = ff.drop(['message'], axis=1)
counts = []
categories = list(df_for_stats.columns.values)
for i in categories:
    count=0
    for x in df_for_stats[i]:
        if x=='T':
            count=count+1
    counts.append((i,count))
            
       #counts.append((i, df_for_stats[i].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number_of_comments'])
df_stats

,category,number_of_comments
0,food,810
1,recharge,450
2,support,371
3,reminders,893
4,travel,2138
5,nearby,807
6,movies,637
7,casual,3282
8,other,1668


In [7]:
print('number of missing messages in the train data:')
df['message'].isnull().sum()
print('number of missing messages in the test data:')
ff['message'].isnull().sum()

number of missing messages in the train data:
number of missing messages in the test data:


0

In [8]:
print(categories)

[u'food', u'recharge', u'support', u'reminders', u'travel', u'nearby', u'movies', u'casual', u'other']


In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"{([^}]*})","",text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [10]:
df['message'] = df['message'].map(lambda com : clean_text(com))


In [11]:
ff['message'] = ff['message'].map(lambda com : clean_text(com))

In [12]:
df.head()

,message,food,recharge,support,reminders,travel,nearby,movies,casual,other
0,7am everyday,F,F,F,T,F,F,F,F,F
1,chocolate cake,T,F,F,F,F,F,F,F,F
2,closed mortice and tenon joint door dimentions,F,F,T,F,F,F,F,F,F
3,train eppo kelambum,F,F,F,F,T,F,F,F,F
4,yesterday i have cancelled the flight ticket,F,F,F,F,T,F,F,F,F


In [13]:
ff.head()

,message,food,recharge,support,reminders,travel,nearby,movies,casual,other
0,nearest metro station,F,F,F,F,F,T,F,F,F
1,pick up n drop service trough cab,F,F,F,F,T,F,F,F,F
2,i wants to buy a bick,F,F,F,F,F,F,F,F,T
3,show me pizza,T,F,F,F,F,F,F,F,F
4,what is the cheapest package to andaman and ni...,F,F,F,F,T,F,F,F,F


In [14]:
X_train = df.message
X_test = ff.message
print(X_train.shape)
print(X_test.shape)

(40659,)
(10000,)


In [29]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

In [30]:
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, df[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_train)
    print('Test accuracy is {}'.format(accuracy_score(df[category], prediction)))
    print('Test precission is {}'.format(precision_score(df[category], prediction,pos_label="T")))
    print('Test recall_score is {}'.format(recall_score(df[category], prediction,pos_label="T")))

... Processing food
Test accuracy is 0.939152463169
Test precission is 0.974165091367
Test recall_score is 0.388539834129
... Processing recharge
Test accuracy is 0.952064733515
Test precission is 0.960355987055
Test recall_score is 0.384515711046
... Processing support
Test accuracy is 0.956909909245
Test precission is 0.961165048544
Test recall_score is 0.0536004331348
... Processing reminders
Test accuracy is 0.962984825008
Test precission is 0.968026992288
Test recall_score is 0.821852407584
... Processing travel
Test accuracy is 0.929904818121
Test precission is 0.941927990708
Test recall_score is 0.840787914692
... Processing nearby
Test accuracy is 0.935684596276
Test precission is 0.953211009174
Test recall_score is 0.452624700501
... Processing movies
Test accuracy is 0.939103273568
Test precission is 0.975716064757
Test recall_score is 0.391358641359
... Processing casual
Test accuracy is 0.922895299934
Test precission is 0.963786531131
Test recall_score is 0.501073492981
...

In [51]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

In [54]:

for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, df[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(ff[category], prediction)))
    print('Test precission is {}'.format(precision_score(ff[category], prediction,pos_label="T")))
    print('Test recall_score is {}'.format(recall_score(ff[category], prediction,pos_label="T")))
    tn, fp, fn, tp=confusion_matrix(ff[category], prediction).ravel()
    print((tn, fp, fn, tp))
    length=len(prediction)
    """for i in range(length):
        if prediction[i]=='T'and ff.iloc[i][category]:
    for i in range(length):
        if prediction[i] != ff.iloc[i][category]:
            print(ff.iloc[i]['message'], 'has been classified as ', prediction[i], 'and should be ', ff.iloc[i][category])
    print('*************completed*************\n\n\n\n\n\n\n\n\n*****************************')"""

... Processing food
Test accuracy is 0.9608
Test precission is 0.862847222222
Test recall_score is 0.613580246914
(9111, 79, 313, 497)
... Processing recharge
Test accuracy is 0.9793
Test precission is 0.817232375979
Test recall_score is 0.695555555556
(9480, 70, 137, 313)
... Processing support
Test accuracy is 0.9715
Test precission is 0.736263736264
Test recall_score is 0.361185983827
(9581, 48, 237, 134)
... Processing reminders
Test accuracy is 0.9766
Test precission is 0.889940828402
Test recall_score is 0.842105263158
(9014, 93, 141, 752)
... Processing travel
Test accuracy is 0.9339
Test precission is 0.831612034127
Test recall_score is 0.866230121609
(7487, 375, 286, 1852)
... Processing nearby
Test accuracy is 0.9494
Test precission is 0.712270803949
Test recall_score is 0.625774473358
(8989, 204, 302, 505)
... Processing movies
Test accuracy is 0.9674
Test precission is 0.832976445396
Test recall_score is 0.610675039246
(9285, 78, 248, 389)
... Processing casual
Test accurac

In [34]:
from sklearn.metrics import *
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, df[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(ff[category], prediction)))
    print('Test precission is {}'.format(precision_score(ff[category], prediction,pos_label="T")))
    print('Test recall_score is {}'.format(recall_score(ff[category], prediction,pos_label="T")))

... Processing food
Test accuracy is 0.9575
Test precission is 0.912205567452
Test recall_score is 0.525925925926
... Processing recharge
Test accuracy is 0.9758
Test precission is 0.866197183099
Test recall_score is 0.546666666667
... Processing support
Test accuracy is 0.9696
Test precission is 0.868131868132
Test recall_score is 0.212938005391
... Processing reminders
Test accuracy is 0.977
Test precission is 0.936758893281
Test recall_score is 0.796192609183
... Processing travel
Test accuracy is 0.9388
Test precission is 0.867888138862
Test recall_score is 0.841908325538
... Processing nearby
Test accuracy is 0.9534
Test precission is 0.798598949212
Test recall_score is 0.565055762082
... Processing movies
Test accuracy is 0.9678
Test precission is 0.922252010724
Test recall_score is 0.540031397174
... Processing casual
Test accuracy is 0.8406
Test precission is 0.940041710115
Test recall_score is 0.549360146252
... Processing other
Test accuracy is 0.9059
Test precission is 0.961